In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np

f = open('IncidenciasTDTGeo.csv', 'r')
cs = csv.reader(f)

data = []
rares = []

for row in cs:
    if row[0] == 'Accidente':
        lat = float(row[-1])
        lon = float(row[-2])
        if lon > -2.25:
            rares.append(row)
        else:
            data.append([lon, lat])

X = np.asarray(data)
            
print 'Accidente:', len(data)
print 'Raros:', len(rares)

Accidente: 7921
Raros: 18


In [2]:
# Plot accidents
fig, ax = plt.subplots()
plt.scatter(X[:,0], X[:,1], c='r', marker='o')
ax.grid(True)
plt.title('Accidentes en Bizkaia')
plt.show()

In [4]:
# Compute k-nearest neighboors
from sklearn.neighbors import DistanceMetric, kneighbors_graph
dist = DistanceMetric.get_metric('euclidean')
matdist = dist.pairwise(X)

min_samples = 12
A = kneighbors_graph(X, min_samples, include_self=False)
Arr = A.toarray()

seq = []
for i in xrange(len(data)):
    for j in xrange(len(data)):
        if Arr[i][j] != 0:
            seq.append(matdist[i][j])
            
seq.sort()
plt.plot(seq)
plt.show()

In [12]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.003, min_samples=min_samples).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
unique_labels = set(db.labels_)
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)

print 'Número de clusters: ', n_clusters

Número de clusters:  149


In [6]:
# Validation/Evaluation
from sklearn import metrics
print "Silhoutte Coefficient: %0.3f" % metrics.silhouette_score(X, db.labels_)

Silhoutte Coefficient: 0.621


In [14]:
#plt.scatter(X[:,0], X[:,1], c=labels)
#plt.show()
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
for k, col in zip(unique_labels, colors):
    if k != -1:
        class_member_mask = (db.labels_ == k)
        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=7)
        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=2)
plt.title('Estimated number of clusters: %d' % n_clusters)
plt.show()